In [15]:
import pandas as pd

df = pd.DataFrame()

df_total = pd.DataFrame()
for i in range(1,4,1):
    file_path = './data/{}.csv'.format(i)
    df = pd.read_csv(file_path, encoding='utf-8')
    df_total = pd.concat([df_total, df])

In [16]:
df_total.columns=['다발구역버전','다발구역아이디','다발구역명','시군구코드','시군구명','사고건수','중심X','중심Y','구역반경','?']

In [17]:
df_total

,다발구역버전,다발구역아이디,다발구역명,시군구코드,시군구명,사고건수,중심X,중심Y,구역반경,?
0,2021090301,1171000504,더청춘팩토리,11710,서울특별시 송파구,4,127.083076,37.506157,40,NaN
1,2021090301,1171000505,의료법인정애의료재단서울초이스병원,11710,서울특별시 송파구,5,127.121734,37.533760,40,NaN
2,2021090301,1171000506,풍납극동아파트,11710,서울특별시 송파구,7,127.116861,37.526038,40,NaN
3,2021090301,1171000507,GS25S8강동구청역점,11710,서울특별시 송파구,4,127.120606,37.530674,40,NaN
4,2021090301,1171000508,남한강올갱이,11710,서울특별시 송파구,9,127.110715,37.510812,40,NaN
...,...,...,...,...,...,...,...,...,...,...
240,2021090301,1129000182,위니아딤채,11290,서울특별시 성북구,8,127.012689,37.591435,40,19.0
241,2021090301,1129000183,제일사,11290,서울특별시 성북구,3,127.021402,37.601773,40,20.0
242,2021090301,1129000184,미니스톱,11290,서울특별시 성북구,11,127.017784,37.593502,40,21.0
243,2021090301,1129000185,들꽃이한의원,11290,서울특별시 성북구,3,127.053216,37.608757,40,22.0


In [19]:
df_total['시군구코드'].unique()

array([11710, 11470, 11560, 11170, 11380, 11110, 11140, 11260, 11680,
       11740, 11305, 11500, 11620, 11215, 11530, 11545, 11350, 11320,
       11230, 11590, 11440, 11410, 11650, 11200, 11290], dtype=int64)

In [25]:
df_total = df_total[df_total['다발구역아이디'].duplicated()==False]

In [26]:
df_total

,다발구역버전,다발구역아이디,다발구역명,시군구코드,시군구명,사고건수,중심X,중심Y,구역반경,?
0,2021090301,1171000504,더청춘팩토리,11710,서울특별시 송파구,4,127.083076,37.506157,40,NaN
1,2021090301,1171000505,의료법인정애의료재단서울초이스병원,11710,서울특별시 송파구,5,127.121734,37.533760,40,NaN
2,2021090301,1171000506,풍납극동아파트,11710,서울특별시 송파구,7,127.116861,37.526038,40,NaN
3,2021090301,1171000507,GS25S8강동구청역점,11710,서울특별시 송파구,4,127.120606,37.530674,40,NaN
4,2021090301,1171000508,남한강올갱이,11710,서울특별시 송파구,9,127.110715,37.510812,40,NaN
...,...,...,...,...,...,...,...,...,...,...
240,2021090301,1129000182,위니아딤채,11290,서울특별시 성북구,8,127.012689,37.591435,40,19.0
241,2021090301,1129000183,제일사,11290,서울특별시 성북구,3,127.021402,37.601773,40,20.0
242,2021090301,1129000184,미니스톱,11290,서울특별시 성북구,11,127.017784,37.593502,40,21.0
243,2021090301,1129000185,들꽃이한의원,11290,서울특별시 성북구,3,127.053216,37.608757,40,22.0


In [27]:
df_total.to_csv('./data/이륜차사고다발지역.csv')